In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/butterfly-image-classification/Training_set.csv
/kaggle/input/butterfly-image-classification/Testing_set.csv
/kaggle/input/butterfly-image-classification/test/Image_747.jpg
/kaggle/input/butterfly-image-classification/test/Image_561.jpg
/kaggle/input/butterfly-image-classification/test/Image_345.jpg
/kaggle/input/butterfly-image-classification/test/Image_2566.jpg
/kaggle/input/butterfly-image-classification/test/Image_1593.jpg
/kaggle/input/butterfly-image-classification/test/Image_844.jpg
/kaggle/input/butterfly-image-classification/test/Image_1975.jpg
/kaggle/input/butterfly-image-classification/test/Image_270.jpg
/kaggle/input/butterfly-image-classification/test/Image_526.jpg
/kaggle/input/butterfly-image-classification/test/Image_968.jpg
/kaggle/input/butterfly-image-classification/test/Image_2104.jpg
/kaggle/input/butterfly-image-classification/test/Image_1769.jpg
/kaggle/input/butterfly-image-classification/test/Image_2008.jpg
/kaggle/input/butterfly-image-classific

In [11]:
!pip install imutils 

In [12]:
from imutils import paths
import os
list_train = list(paths.list_images('/kaggle/input/butterfly-image-classification/train'))
list_test = list(paths.list_images('/kaggle/input/butterfly-image-classification/test'))
train_set = pd.read_csv('/kaggle/input/butterfly-image-classification/Training_set.csv')
test_set = pd.read_csv('/kaggle/input/butterfly-image-classification/Testing_set.csv')

In [13]:
print(len(list_train))
print(len(list_test))

6499
2786


In [14]:
print(train_set.head())
print(train_set.shape)

      filename                     label
0  Image_1.jpg          SOUTHERN DOGFACE
1  Image_2.jpg                    ADONIS
2  Image_3.jpg            BROWN SIPROETA
3  Image_4.jpg                   MONARCH
4  Image_5.jpg  GREEN CELLED CATTLEHEART
(6499, 2)


In [15]:
print(test_set.head())
print(test_set.shape)

      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg
(2786, 1)


# Train models

## Model1 SVM

In [16]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder

In [17]:
train1 = []
labels1 = []
for i in list_train:
    label = train_set[train_set['filename']==i.split(os.path.sep)[-1]]['label'].values[0]
    img = load_img(i,target_size = (32,32,3))
    img = img_to_array(img)
    img = img.flatten()
    train1.append(img)
    labels1.append(label)
train1 = np.array(train1,dtype = 'float32')  
labels1 = np.array(labels1)
enc = LabelEncoder().fit(labels1)
labels1 = enc.transform(labels1)

In [18]:
test_data1 = []
test_label = []
for i in list_test:
    label = i.split(os.path.sep)[-1]
    img = load_img(i,target_size = (32,32,3))
    img = img_to_array(img)
    img = img.flatten()
    test_data1.append(img)
    test_label.append(label)
test_data1 = np.array(test_data1,dtype='float32')/255
test_label = np.array(test_label)

In [19]:
test_label = pd.DataFrame({'filename':test_label})

In [20]:
print(train1[:5])
print(labels1[:5])

[[ 30.  42.  18. ...  24.  31.  15.]
 [118. 114.  69. ... 187. 212. 110.]
 [157. 161.  38. ... 231. 220. 200.]
 [174. 149. 118. ...  85.  81.  80.]
 [119. 155.  68. ... 166. 205. 124.]]
[20 21 11  8 33]


In [21]:
df1 = pd.DataFrame(train1)
df1['target'] = labels1
x = df1.iloc[:,:-1]/255
y = df1.iloc[:,-1]

In [22]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.117647,0.164706,0.070588,0.113725,0.160784,0.066667,0.109804,0.156863,0.070588,0.101961,...,0.050980,0.086275,0.113725,0.050980,0.090196,0.117647,0.054902,0.094118,0.121569,0.058824
1,0.462745,0.447059,0.270588,0.466667,0.435294,0.290196,0.458824,0.419608,0.282353,0.454902,...,0.482353,0.827451,0.909804,0.552941,0.745098,0.831373,0.439216,0.733333,0.831373,0.431373
2,0.615686,0.631373,0.149020,0.619608,0.639216,0.145098,0.643137,0.654902,0.149020,0.654902,...,0.741176,0.937255,0.858824,0.760784,0.862745,0.784314,0.686275,0.905882,0.862745,0.784314
3,0.682353,0.584314,0.462745,0.854902,0.784314,0.776471,0.784314,0.713726,0.698039,0.580392,...,0.062745,0.705882,0.690196,0.694118,0.572549,0.545098,0.572549,0.333333,0.317647,0.313726
4,0.466667,0.607843,0.266667,0.478431,0.615686,0.286275,0.482353,0.623529,0.282353,0.470588,...,0.121569,0.435294,0.545098,0.254902,0.607843,0.745098,0.415686,0.650980,0.803922,0.486275


In [23]:
from sklearn.model_selection import train_test_split
X_train1,X_valid1,y_train1,y_valid1 = train_test_split(x, y, test_size = 0.2, random_state = 27,stratify = y) 

In [24]:
pars = {'C' : [100],
       'gamma' : [0.0001],
       'kernel' : ['rbf']}
svc = svm.SVC(probability = True)
model1 = GridSearchCV(svc,pars,verbose=10)

In [25]:
model1.fit(X_train1,y_train1)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 1/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.338 total time= 3.2min
[CV 2/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 2/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.362 total time= 3.3min
[CV 3/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 3/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.346 total time= 3.2min
[CV 4/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 4/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.330 total time= 3.2min
[CV 5/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 5/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.341 total time= 3.2min


GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [100], 'gamma': [0.0001], 'kernel': ['rbf']},
             verbose=10)

In [26]:
pred1 = model1.predict(X_valid1)

In [27]:
print(model1.best_params_)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}


In [28]:
print(pred1[:5])

[11 17 41 10 54]


In [29]:
print(accuracy_score(y_valid1,pred1))
print(classification_report(y_valid1,pred1))

0.3446153846153846
              precision    recall  f1-score   support

           0       0.71      0.56      0.63        18
           1       0.82      0.60      0.69        15
           2       0.10      0.13      0.11        15
           3       0.62      0.29      0.40        17
           4       0.48      0.67      0.56        18
           5       0.23      0.45      0.31        20
           6       0.32      0.50      0.39        20
           7       0.28      0.29      0.29        17
           8       0.19      0.25      0.22        16
           9       0.14      0.24      0.18        17
          10       0.57      0.53      0.55        15
          11       0.14      0.18      0.16        17
          12       0.28      0.25      0.26        20
          13       0.32      0.44      0.37        18
          14       0.53      0.59      0.56        17
          15       0.27      0.37      0.31        19
          16       0.36      0.24      0.29        17
        

In [30]:
pred_test1 = model1.predict_proba(test_data1)

## Model KNN

In [32]:
from sklearn.neighbors import KNeighborsClassifier
par2 = {'n_neighbors': [1,2]}
kneig = KNeighborsClassifier()
model2 = GridSearchCV(kneig,par2,verbose=10)
model2.fit(X_train1,y_train1)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5; 1/2] START n_neighbors=1...............................................
[CV 1/5; 1/2] END ................n_neighbors=1;, score=0.167 total time=   0.9s
[CV 2/5; 1/2] START n_neighbors=1...............................................
[CV 2/5; 1/2] END ................n_neighbors=1;, score=0.172 total time=   0.8s
[CV 3/5; 1/2] START n_neighbors=1...............................................
[CV 3/5; 1/2] END ................n_neighbors=1;, score=0.155 total time=   0.7s
[CV 4/5; 1/2] START n_neighbors=1...............................................
[CV 4/5; 1/2] END ................n_neighbors=1;, score=0.161 total time=   0.7s
[CV 5/5; 1/2] START n_neighbors=1...............................................
[CV 5/5; 1/2] END ................n_neighbors=1;, score=0.167 total time=   0.7s
[CV 1/5; 2/2] START n_neighbors=2...............................................
[CV 1/5; 2/2] END ................n_neighbors=2;,

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2]}, verbose=10)

In [33]:
pred2 = model2.predict(X_valid1)

In [ ]:
print(model2.best_params_)

In [34]:
print(pred2[:5])

[12 41 64 10 54]


In [35]:
print(accuracy_score(y_valid1,pred2))
print(classification_report(y_valid1,pred2))

0.16307692307692306
              precision    recall  f1-score   support

           0       0.86      0.33      0.48        18
           1       0.67      0.40      0.50        15
           2       0.00      0.00      0.00        15
           3       1.00      0.12      0.21        17
           4       0.00      0.00      0.00        18
           5       0.28      0.25      0.26        20
           6       0.62      0.25      0.36        20
           7       0.50      0.18      0.26        17
           8       0.07      0.12      0.09        16
           9       0.03      0.24      0.06        17
          10       0.50      0.40      0.44        15
          11       0.07      0.18      0.10        17
          12       0.23      0.15      0.18        20
          13       0.22      0.22      0.22        18
          14       0.50      0.18      0.26        17
          15       0.06      0.16      0.09        19
          16       0.67      0.12      0.20        17
       

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
pred_test2 = model2.predict_proba(test_data1)

## Model Random Forest

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
par3 = {'n_estimators':[500],
       'max_features':['sqrt'],
       'max_depth':[19],
       'criterion':['gini']}

In [39]:
rfc = RandomForestClassifier(random_state=27)
model3 = GridSearchCV(rfc,par3,verbose=10)
model3.fit(X_train1,y_train1)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 1/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.364 total time= 3.2min
[CV 2/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 2/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.365 total time= 3.2min
[CV 3/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 3/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.348 total time= 3.1min
[CV 4/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 4/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.358 total time= 3.2min
[CV 5/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 5/5; 1/1] END criterion=gini, max_depth

GridSearchCV(estimator=RandomForestClassifier(random_state=27),
             param_grid={'criterion': ['gini'], 'max_depth': [19],
                         'max_features': ['sqrt'], 'n_estimators': [500]},
             verbose=10)

In [40]:
pred3 = model3.predict(X_valid1)

In [41]:
print(pred3[:5])

[12 17 41 70 54]


In [42]:
print(model3.best_params_)

{'criterion': 'gini', 'max_depth': 19, 'max_features': 'sqrt', 'n_estimators': 500}


In [43]:
print(accuracy_score(y_valid1,pred3))
print(classification_report(y_valid1,pred3))

0.3384615384615385
              precision    recall  f1-score   support

           0       0.45      0.56      0.50        18
           1       0.50      0.60      0.55        15
           2       0.25      0.07      0.11        15
           3       0.75      0.35      0.48        17
           4       0.59      0.56      0.57        18
           5       0.22      0.60      0.32        20
           6       0.36      0.60      0.45        20
           7       0.62      0.29      0.40        17
           8       0.11      0.12      0.11        16
           9       0.15      0.47      0.23        17
          10       0.80      0.27      0.40        15
          11       0.00      0.00      0.00        17
          12       0.28      0.25      0.26        20
          13       0.38      0.56      0.45        18
          14       0.50      0.41      0.45        17
          15       0.10      0.11      0.10        19
          16       0.20      0.06      0.09        17
        

In [44]:
pred_test3 = model3.predict_proba(test_data1)

## Model Ensembled

In [ ]:
#pred123 = (pred_test1 + pred_test2 + pred_test3)/3

In [ ]:
#pred123 = np.argmax(pred123,axis = 1)

In [ ]:
#test_label['label'] = enc.inverse_transform(pred123)

In [ ]:
#submission = pd.merge(test_set,test_label,how = 'left',on = 'filename')

In [ ]:
#submission.to_csv('Testing_set.csv',index = False)

In [ ]:
#submission.head()

## Model CNN DenseNet121

In [45]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
train_data = []
train_label = []
for i in list_train:
    label = train_set[train_set['filename']==i.split(os.path.sep)[-1]]['label'].values[0]
    image = load_img(i,target_size = (224,224,3))
    image = img_to_array(image)
    train_data.append(image)
    train_label.append(label)
train_data = np.array(train_data,dtype='float32')
train_label = np.array(train_label)  

In [46]:
test_data = []
test_label = []
for i in list_test:
    label = i.split(os.path.sep)[-1]
    image = load_img(i,target_size = (224,224,3))
    image = img_to_array(image)
    test_data.append(image)
    test_label.append(label)
test_data = np.array(test_data,dtype='float32')
test_label = np.array(test_label)

In [47]:
test_label = pd.DataFrame({'filename':test_label})

In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [49]:
enc = LabelEncoder()
enc.fit(train_label)
train_label = enc.transform(train_label)
print(train_label[:2])

[20 21]


In [50]:
train_label = tf.keras.utils.to_categorical(train_label,num_classes = 75)
print(train_label[:2])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.2, random_state = 27, stratify = train_label)

In [52]:
print(len(X_train))
print(len(X_valid))
print(len(y_train))
print(len(y_valid))

5199
1300
5199
1300


In [53]:
gen = ImageDataGenerator(zoom_range = 0.2,
                        height_shift_range = 0.2,
                        width_shift_range = 0.2,
                        rotation_range = 20,
                        shear_range = 0.2,
                        horizontal_flip = True,
                        vertical_flip = True,
                        fill_mode = 'nearest'
                        )

In [54]:
basemodel1 = DenseNet121(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel1 = basemodel1.output
headmodel1 = AveragePooling2D()(headmodel1)
headmodel1 = Flatten()(headmodel1)
headmodel1 = Dense(units=512,activation = 'relu')(headmodel1)
headmodel1 = Dropout(rate = 0.5)(headmodel1)
headmodel1 = Dense(units=128,activation = 'relu')(headmodel1)
headmodel1 = Dense(units = 75,activation = 'softmax')(headmodel1)
model_densenet = Model(inputs = basemodel1.input, outputs = headmodel1)

29084464/29084464 [==============================] - 2s 0us/step


In [55]:
for i in basemodel1.layers:
    i.trainable = False

In [56]:
lr = 0.0001
Epochs = 8
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model_densenet.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = [keras.metrics.AUC(name = 'auc')])
model_densenet.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

Epoch 1/8
81/81 [==============================] - 714s 9s/step - loss: 7.0060 - auc: 0.5247 - val_loss: 4.2486 - val_auc: 0.5963
Epoch 2/8
81/81 [==============================] - 713s 9s/step - loss: 4.3977 - auc: 0.5771 - val_loss: 4.1874 - val_auc: 0.6060
Epoch 3/8
81/81 [==============================] - 707s 9s/step - loss: 4.2526 - auc: 0.5919 - val_loss: 4.1473 - val_auc: 0.6204
Epoch 4/8
81/81 [==============================] - 708s 9s/step - loss: 4.1891 - auc: 0.6211 - val_loss: 4.0349 - val_auc: 0.6648
Epoch 5/8
81/81 [==============================] - 697s 9s/step - loss: 4.1054 - auc: 0.6477 - val_loss: 3.9217 - val_auc: 0.6972
Epoch 6/8
81/81 [==============================] - 710s 9s/step - loss: 4.0270 - auc: 0.6731 - val_loss: 3.8304 - val_auc: 0.7220
Epoch 7/8
81/81 [==============================] - 720s 9s/step - loss: 3.9289 - auc: 0.6987 - val_loss: 3.7000 - val_auc: 0.7476
Epoch 8/8
81/81 [==============================] - 713s 9s/step - loss: 3.8267 - auc: 0.72

## Model CNN EfficientNetB0

In [57]:
basemodel = EfficientNetB0(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel = basemodel.output
headmodel = AveragePooling2D()(headmodel)
headmodel = Flatten()(headmodel)
headmodel = Dense(units=512,activation = 'relu')(headmodel)
headmodel = Dropout(rate = 0.5)(headmodel)
headmodel = Dense(units=128,activation = 'relu')(headmodel)
headmodel = Dense(units = 75,activation = 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)

16705208/16705208 [==============================] - 1s 0us/step


In [58]:
for i in basemodel.layers:
    i.trainable = False

In [59]:
lr = 0.0001
Epochs = 8
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = [keras.metrics.AUC(name = 'auc')])
model.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

Epoch 1/8
81/81 [==============================] - 373s 4s/step - loss: 4.1372 - auc: 0.6527 - val_loss: 3.5334 - val_auc: 0.8482
Epoch 2/8
81/81 [==============================] - 371s 5s/step - loss: 3.2317 - auc: 0.8485 - val_loss: 2.2233 - val_auc: 0.9575
Epoch 3/8
81/81 [==============================] - 370s 5s/step - loss: 2.3466 - auc: 0.9332 - val_loss: 1.3894 - val_auc: 0.9839
Epoch 4/8
81/81 [==============================] - 368s 5s/step - loss: 1.7435 - auc: 0.9636 - val_loss: 1.0221 - val_auc: 0.9874
Epoch 5/8
81/81 [==============================] - 371s 5s/step - loss: 1.4111 - auc: 0.9760 - val_loss: 0.7894 - val_auc: 0.9909
Epoch 6/8
81/81 [==============================] - 354s 4s/step - loss: 1.2153 - auc: 0.9797 - val_loss: 0.6877 - val_auc: 0.9915
Epoch 7/8
81/81 [==============================] - 357s 4s/step - loss: 1.0500 - auc: 0.9837 - val_loss: 0.6111 - val_auc: 0.9926
Epoch 8/8
81/81 [==============================] - 369s 5s/step - loss: 0.9625 - auc: 0.98

## Model CNN EfficientNetV2**

In [60]:
basemodel2 = EfficientNetV2L(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel2 = basemodel2.output
headmodel2 = AveragePooling2D()(headmodel2)
headmodel2 = Flatten()(headmodel2)
headmodel2 = Dense(units=128,activation = 'relu')(headmodel2)
headmodel2 = Dropout(rate = 0.5)(headmodel2)
headmodel2 = Dense(units=128,activation = 'relu')(headmodel2)
headmodel2 = Dense(units = 75,activation = 'softmax')(headmodel2)
model_v2 = Model(inputs = basemodel2.input, outputs = headmodel2)

473176280/473176280 [==============================] - 19s 0us/step


In [61]:
for i in basemodel2.layers:
    i.trainable = False

In [62]:
lr = 0.0001
Epochs = 8
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model_v2.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = [keras.metrics.AUC(name = 'auc')])
model_v2.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

Epoch 1/8
81/81 [==============================] - 2536s 31s/step - loss: 4.2377 - auc: 0.5901 - val_loss: 3.9885 - val_auc: 0.6896
Epoch 2/8
81/81 [==============================] - 2440s 30s/step - loss: 3.9750 - auc: 0.6949 - val_loss: 3.6625 - val_auc: 0.7813
Epoch 3/8
81/81 [==============================] - 2477s 31s/step - loss: 3.7307 - auc: 0.7606 - val_loss: 3.3941 - val_auc: 0.8453
Epoch 4/8
81/81 [==============================] - 2464s 30s/step - loss: 3.4901 - auc: 0.8149 - val_loss: 3.1216 - val_auc: 0.8901
Epoch 5/8
81/81 [==============================] - 2441s 30s/step - loss: 3.2525 - auc: 0.8540 - val_loss: 2.8476 - val_auc: 0.9175
Epoch 6/8
81/81 [==============================] - 2471s 31s/step - loss: 3.0844 - auc: 0.8787 - val_loss: 2.6879 - val_auc: 0.9300
Epoch 7/8
81/81 [==============================] - 2477s 31s/step - loss: 2.8842 - auc: 0.9023 - val_loss: 2.5145 - val_auc: 0.9412
Epoch 8/8
81/81 [==============================] - 2454s 30s/step - loss: 2.

# Test data

In [ ]:
# = model.predict(test_data)

In [ ]:
#print(predict[:10])

In [ ]:
#predict = np.argmax(predict,axis = 1)

In [ ]:
#predict[:5]

In [ ]:
#test_label['label'] = enc.inverse_transform(predict)

In [ ]:
#test_label

In [ ]:
#test_label[test_label['filename']=='Image_1.jpg']

In [ ]:
#submission = pd.merge(test_set,test_label,how = 'left',on = 'filename')

In [ ]:
#print(test_set.shape)
#print(submission.shape)

In [ ]:
#submission.head()

In [ ]:
#submission.to_csv('Testing_set.csv',index = False)

In [63]:
predict1 = 0.2 * model_v2.predict(test_data) + 0.6 * model.predict(test_data) \
+ 0.05 * model_densenet.predict(test_data) + 0.05 * model1.predict_proba(test_data1) \
+ 0.05 * model2.predict_proba(test_data1) + 0.05 * model3.predict_proba(test_data1)
predict1 = np.argmax(predict1,axis = 1)
test_label['label'] = enc.inverse_transform(predict1)
submission1 = pd.merge(test_set,test_label,how = 'left',on = 'filename')
submission1.to_csv('Testing_set1.csv',index = False)

88/88 [==============================] - 254s 3s/step
